In [1]:
import boto3
import json

In [2]:
s3 = boto3.client("s3", region_name='us-west-2')

response = s3.get_object(
    Bucket='udacity-demo-asdfg',
    Key='Apollo13.txt'
)

apollo13 = response['Body'].read().decode('utf-8')

response = s3.get_object(
    Bucket='udacity-demo-asdfg',
    Key='transcript.txt'
)

transcript = response['Body'].read().decode('utf-8')


In [3]:
client = boto3.client('bedrock-runtime', region_name='us-west-2')

In [4]:
model_id = "anthropic.claude-3-5-sonnet-20241022-v2:0"

In [5]:
prompt_text = f"""
<context>
{apollo13}
</context>

<instruction>
You are to give me a bullet point summary of what happened during the transctip.
As well, please provide a sentiment analysis of the transcript.
Please provide your response in a valid JSON format with the following keys:
- summary: a bullet point summary of what happened during the transcript
- sentiment: a sentiment analysis of the transcript
</instruction>

<text>
{transcript}
</text>
"""

In [6]:
# Format the request payload using the model's native structure
request_body = {
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 1024,
    "temperature": 1,
    'top_p': 0.999,
    "messages": [
        {
            "role": "user",
            "content": [{"type": "text", "text": prompt_text}]
        }
    ]
}

In [7]:
# Invoke the model
response = client.invoke_model(
    modelId=model_id,
    body=json.dumps(request_body)
)

In [8]:
# Parse the response
response_body = json.loads(response['body'].read())
generated_text = response_body['content'][0]['text']
print(generated_text)

{
    "summary": [
        "- The transcript captures communications during a critical failure on Apollo 13",
        "- Multiple system failures occurred, including oxygen tank problems and power issues",
        "- The crew reported hearing a 'bang' followed by warning lights and system malfunctions",
        "- Mission Control guided the crew through various power-down procedures",
        "- Fuel cells 1 and 3 failed, leaving only fuel cell 2 operational",
        "- The crew observed venting of gas into space through window 1",
        "- Multiple thruster reconfigurations were necessary to maintain spacecraft stability",
        "- Mission Control began planning for alternative mission scenarios using LM power",
        "- Continuous monitoring of oxygen levels and power consumption became critical"
    ],
    "sentiment": {
        "overall": "tense and urgent",
        "key_aspects": [
            "High stress levels evident in communications",
            "Professional but con